In [60]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
from torch import nn
from torch.utils import data

关于下标
1. 0~N-1：琪琪的属性嵌入、匹配度、满意度
2. 1~N：whc做的那两个dict文件
3. 真实ID：导出文件格式（题目要求）

In [2]:
# ID映射表
# df_per = pd.read_csv('./user_dict',sep='\t')
# df_rec = pd.read_csv('./jd_dict',sep='\t')

In [3]:
# 匹配度、满意度，这里的序号是0~N-1
df_ppd = pd.read_csv('./p-u-rating.txt',sep=' ')
df_myd = pd.read_csv('./u-p-rating.txt',sep=' ')
df_ppd.head(), df_myd.head()

(   position  user  rating
 0         0    94       6
 1         0   165       6
 2         0   168       6
 3         1    94       6
 4         1   165       6,
    user  position  myd
 0   425       295    8
 1   425      1504    8
 2   425      1515    8
 3   425      1542    8
 4   425      1543    8)

In [4]:
def is_fw(pid):
  # return not pid in idx
  # print(f'is_fw({pid})')
  val = fw_list.loc[pid,'是否为废人']
  return val==1

fw_list = pd.read_csv('./fw.txt',sep=' ',index_col=0)
fw_list.head()

,是否为废人
下标,
0,0
1,1
2,1
3,1
4,1


In [5]:
alive = []
for i in range(10877):
  if is_fw(i):
    pass
  else:
    alive.append(i)

In [6]:
for i in range(74):
  print(f'{i} ',end='')

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 

In [7]:
# df_map.to_csv('./per_job_map.csv',header=['job','person','label'],index=None)

In [8]:
# 属性嵌入，这里的序号是0~N-1
a_emb_job = pd.read_csv('./attribute_embedding_position.txt',sep=' ',index_col=0,header=None,na_values=[])
a_emb_per = pd.read_csv('./attribute_embedding_user.txt',sep=' ',index_col=0,header=None,na_values=[])
a_emb_per.index = [alive[i] for i in range(868)]
print(a_emb_job.shape, a_emb_per.shape)
a_emb_per.head()

(1575, 74) (868, 74)


,1,2,3,4,5,6,7,8,9,10,...,65,66,67,68,69,70,71,72,73,74
0,-0.724494,-0.559137,-0.300785,0.008680,-0.170450,0,0,0.0,1,0,...,-0.160764,0.035278,0.060169,0.088794,-0.306723,0.137945,0.316116,0.4042,0.127718,0.047178
7,-0.724494,-0.721122,-0.890788,4.934121,3.222017,0,0,0.0,1,0,...,-0.160764,0.035278,0.060169,0.088794,-0.306723,0.137945,0.316116,0.4042,0.127718,0.047178
10,1.741550,-0.802114,-0.890788,2.832705,7.543888,0,0,0.0,0,1,...,-0.160764,0.035278,0.060169,0.088794,-0.306723,0.137945,0.316116,0.4042,0.127718,0.047178
12,-0.724494,-0.397152,-0.448285,0.080217,-0.179261,0,0,0.0,1,1,...,-0.160764,0.035278,0.060169,0.088794,-0.306723,0.137945,0.316116,0.4042,0.127718,0.047178
13,-0.724494,3.247509,6.041750,0.025324,-0.260802,0,0,0.0,1,1,...,-0.160764,0.035278,0.060169,0.088794,-0.306723,0.137945,0.316116,0.4042,0.127718,0.047178


先算PU网络，得到匹配度

In [9]:
# 结构嵌入，这里的序号是0~N-1
d_emb_job = pd.read_csv('./position_up.csv',index_col=0,header=0,na_values=[])
d_emb_per = pd.read_csv('./user_up.csv',index_col=0,header=0,na_values=[])
d_emb_per.index = [alive[i] for i in range(868)]
print(d_emb_job.shape, d_emb_per.shape)
d_emb_per.head()

(1575, 128) (868, 128)


,1,2,3,4,5,6,7,8,9,10,...,119,120,121,122,123,124,125,126,127,128
0,-0.051832,-0.149874,-0.224147,0.011519,0.053820,0.264336,0.022562,0.261905,-0.074125,-0.005507,...,0.012419,-0.086074,0.078442,0.012039,-0.055958,0.038050,0.034189,-0.160455,0.049304,0.076718
7,-0.766611,0.110375,-0.580904,-0.090295,0.201381,-0.176775,0.435382,0.769255,0.402643,-0.218459,...,-0.373447,-0.226740,-0.250924,0.050345,-0.059023,0.161022,-0.184423,0.433543,-0.437071,-0.446206
10,-0.819879,0.164693,-0.661215,-0.131313,0.257609,-0.147225,0.528567,0.879149,0.421028,-0.252310,...,-0.387897,-0.269366,-0.234926,0.047197,-0.048544,0.149407,-0.166082,0.490235,-0.481465,-0.484754
12,-0.095432,-0.154287,-0.198380,0.020648,0.023234,0.235723,-0.019982,0.245697,-0.070095,0.051449,...,0.014524,-0.051286,0.081739,0.013149,-0.091155,0.068120,0.025149,-0.182351,0.108150,0.082967
13,0.088222,-0.003400,-0.018967,0.089848,0.040171,-0.089441,0.054320,0.035361,-0.083049,-0.068331,...,0.051633,-0.029428,0.059547,-0.065758,-0.069571,0.088358,0.039683,-0.072314,0.047281,0.045774


In [10]:
# a = np.array([[1,2,3],[4,5,6]])
# b = np.array([[-1,-2],[-3,-4]])
a = a_emb_per.values
b = d_emb_per.values
X = np.concatenate([a,b],axis=1)

c = a_emb_job.values
d = d_emb_job.values
Y = np.concatenate([c,d],axis=1)
X.shape, Y.shape

((868, 202), (1575, 202))

In [11]:
import sklearn
from sklearn.decomposition import PCA
pca=PCA(n_components=64)
pca.fit(X)
Xpca = pca.transform(X) #进行降维
pca.fit(Y)
Ypca = pca.transform(Y)

In [12]:
Xpca.shape, Ypca.shape

((868, 64), (1575, 64))

In [13]:
pca_per = pd.DataFrame(Xpca, index=[alive[i] for i in range(868)])
pca_job = pd.DataFrame(Ypca)
pca_per.shape,pca_job.shape

((868, 64), (1575, 64))

In [14]:
def get_input(pid, rid):
  per = pca_per.loc[pid].values
  job = pca_job.loc[rid].values
  all = np.concatenate([per,job],axis=0)
  return all

In [15]:
features = []
labels = []
# get_input(7,1)

In [16]:
for i in tqdm(range(df_myd.shape[0])):
  row = df_myd.loc[i]
  pid = row['user']
  rid = row['position']
  label = row['myd']
  if is_fw(pid):
    continue
  # print(pid,rid,label)
  feature = get_input(pid,rid)
  features.append(feature)
  labels.append(label)
  

100%|████████████████████████████████████████████████████████████████████████| 433890/433890 [01:03<00:00, 6785.40it/s]


In [17]:
features = np.array(features)
labels = np.array(labels)
features = torch.Tensor(features)
labels = torch.Tensor(labels).reshape(-1,1)
features.shape,labels.shape

(torch.Size([433890, 128]), torch.Size([433890, 1]))

In [18]:
class DS:
  def __init__(self,data_arrays):
    self.data = data_arrays

  def __getitem__(self,index):
    return self.data[0][index],self.data[1][index]
  
  def __len__(self):
    return self.data[0].size()[0]

def load_array(data_arrays, batch_size, is_train=True):
  # Array to Iter
  dataset = data.TensorDataset(*data_arrays)
  return data.DataLoader(dataset, batch_size, shuffle=is_train)
    

In [50]:
batch_size = 128

In [51]:
train_iter = load_array((features,labels), batch_size)
# test_iter = load_array((features[train_size:train_size+test_size],labels[train_size:train_size+test_size]), batch_size, False)

In [52]:
net = nn.Sequential(nn.Linear(128,32),
                    nn.ReLU(),
                    nn.Linear(32,1)
                    )

In [61]:
net = torch.load('./DNN-MYD.pt')

In [53]:
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

net.apply(init_weights)

Sequential(
  (0): Linear(in_features=128, out_features=32, bias=True)
  (1): ReLU()
  (2): Linear(in_features=32, out_features=1, bias=True)
)

In [54]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [64]:
net = net.to("cpu")
# loss = nn.MSELoss().to(device)
loss = nn.MSELoss()
# loss = nn.CrossEntropyLoss(reduction='none')
trainer = torch.optim.Adam(net.parameters(), lr=0.01)

In [56]:
total_epochs = 0

In [ ]:
num_epochs = 1000
looper = tqdm(range(num_epochs))
for epoch in looper:
    total_epochs += 1
    for X, y in train_iter:
        # print(X.shape, y.shape)
#         X,y = X.to(device),y.to(device)
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    # print(f'epoch {epoch + 1}, loss {l:f}')
    looper.set_description(f'epoch {total_epochs}, loss {l:f}')

epoch 1985, loss 0.544682:  97%|████████████████████████████████████████████████▋ | 973/1000 [2:01:20<03:14,  7.20s/it]

In [67]:
print(net(torch.Tensor(get_input(0,7)))) # None
print(net(torch.Tensor(get_input(8031,1038)))) # 2
print(net(torch.Tensor(get_input(94,0)))) # 6
# net.state_dict()

tensor([4.1503], grad_fn=<AddBackward0>)
tensor([7.1419], grad_fn=<AddBackward0>)
tensor([4.6116], grad_fn=<AddBackward0>)


In [68]:
torch.save(net,'DNN-MYD.pt')